### 1장 코틀린이란 무엇이며, 왜 필요한가?

작은 예제로 시작해보자. 여기서 코틀린만의 여러 특성을 발견할 수 있다.

1. 클래스 본문 없이도 프로퍼티가 포함된 데이터 클래스를 정의한다.
2. `val` 키워드를 이용해 읽기 전용 프로퍼티를 선언한다.
3. 인자의 기본값을 지정한다.
4. 타입시스템을 이용해 `null` 이 될 수 있는 값들을 명시적으로 다뤄, NPE를 피한다.
5. 클래스 안에 포함될 필요가 없는 **최상위 함수**를 정의한다.
6. 함수나 생성자를 호출할 때 파라미터 이름을 지정한 인자를 사용한다.
7. 트레일링 콤마(trailing comma)를 사용한다.
8. 람다식을 사용해 컬렉션 연산을 활용한다.
9. 엘비스 연산자(`?:`)를 사용해 변수가 `null` 일때에 대비한 값을 제공한다.
10. 수동으로 문자열을 연결하지 않고, 문자열 템플릿을 사용한다.
11. 데이터 클래스를 위해 컴파일러가 자동으로 생성하는 `toString` 같은 함수를 사용한다.

In [15]:
data class Person( //데이터 클래스
    val name: String, //읽기 전용 val 프로퍼티
    val age: Int? = null // null이 될 수 있는 타입과 파라미터의 기본값 지정
)

In [16]:
fun main() { //최상위 함수
    val persons = listOf(
        Person("현진", age = 28), //이름 붙은 파라미터
        Person("병국"), // 트레일링 콤마
    )
    val oldest = persons.maxByOrNull { //람다식
        it.age ?: Int.MIN_VALUE // null에 적용되는 엘비스 연산자
    }
    println("나이가 가장 많은 사람: $oldest") //문자열 템플릿, toString 자동 생성
}

main()

나이가 가장 많은 사람: Person(name=현진, age=28)



람다식의 파라미터가 하나인 경우 임시적인 디폴트 이름 it을 사용할 수 있다.
> 물론 별도로 이름지정도 가능

코틀린은 정적타입언어지만, 대부분의 경우 코틀린 컴파일러가 문맥으로부터 변수 타임을 자동으로 유추할 수 있다. 따라서 다음의 코드가 가능하다.

In [ ]:
val x: Int = 1
val y = 1

컴파일러가 문맥을 고려해 변수 타입을 결정하는 이런 기능을 타입추론(type inference)라고 부름.

또한 코틀린은 특별한 타입으로 '널이 될 수 있는 타입'(nullable type)을 지원한다. 이를 통해 런타임에 NPE가 발생하는지 컴파일 타임에서 검사할 수 있음.

코틀린은 다중 패러다임 언어로 함수형 프로그래밍도 지원한다. 함수형의 핵심개념은 다음과 같다.
1. 일급 시민로 다뤄지는 함수
2. 불변 객체를 이용해 작성되는 코드
3. 부수 효과 없음. 순수 함수 사용

함수형 프로그래밍의 장점중 하나는 코드 중복을 피할 수 있다는 점. 공통 로직을 따로 함수로 뽑아내고 다른 세부사항을 인자로 전달 가능. 이 인자 자체가 함수가 될 수 있음.
코틀린에서 람다를 사용하는 방식이 위의 함수형 프로그래밍 방식

또한 안전한 동시성 제공. 불변 데이터 구조와 순수 함수를 사용하면 안전하지 않은 데이터 변경이 발생하지 않는다고 생각할 수 있다.

다음으로 함수형 프로그래밍은 테스트 하기 쉽다.부수 효과가 없는 함수는 그 함수가 의존하는 전체 환경을 구축하기 위한 준비 코드 없이 독립적으로 테스트 할 수 있다.

### 동시성 처리와 코루틴
프로그래밍 언어들은 동시성 처리를 위해 스레드, 콜백, 퓨처, 프로미스, reactive extension 등 다양한 시도를 했다. 코틀린은 coroutine이라는 접근법을 취함.
코루틴은 코드가 자신의 실행을 잠시 중단시킬 수 있고, 나중에 중단했던 지점에서 작업을 계속 수행할 수 있다.
> 이거 JS나 파이썬의 제네레이터하고 비슷한거 아닌가? JVM이라서 동작 방식이 다를지도

코루틴을 이용하면 외부 API 요청 등 오래 걸리는 작업을 진행할 때도 해당 요청을 호출한 스레드를 블락하지 않음.

In [20]:
@file:Repository("https://repo1.maven.org/maven2")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.9.0")

import kotlinx.coroutines.async
import kotlinx.coroutines.coroutineScope
import kotlinx.coroutines.runBlocking

fun loadImage(url: String): String = "$url"
fun combineImages(img1: String, img2: String): String = "Both images are $img1 and $img2 combined"

suspend fun loadAndOverlay(first: String, second: String): String =
    coroutineScope {
        val firstDeferred = async { loadImage(first) }
        val secondDeffered = async { loadImage(second) }
        combineImages(firstDeferred.await(), secondDeffered.await())
    }

runBlocking {
    loadAndOverlay("https://example.com/img1.jpg", "https://example.com/img2.jpg")
 }

Both images are https://example.com/img1.jpg and https://example.com/img2.jpg

> 코틀린에서 'suspend' 키워드를 사용한 함수는 코틀린 내에서만 호출 가능. 그래서 runBlocking 블록 안에서 호출했음.

### 다중 플랫폼
코틀린은 JVM밖에서도 돌아간다.
1. 코틀린을 JS로 컴파일할 수도 있다. 이 말은 코틀린을 브라우저나 노드에서도 돌릴 수 있다는 의미.
2. 코틀린/네이티브를 사용하면 코틀린을 바로 이진 코드로 컴파일할 수 있다. JVM에 의존하지 않을 수도 있음.
3. 코틀린/Wasm은 코틀린을 웹 어셈블리 바이너리로 컴파일할 수 있다.

또한 나중에 더 다루겠지만, `expect` 와 `actual` 키워드를 사용해 플랫폼별 구현을 분리할 수도 있다.
> 이게 모바일쪽에서는 비즈니스로직을 그대로 두면서도, ios와 안드로이드의 별도 네이티브 코드 작성에 사용되는 듯?